In [1]:
import os
import pickle
import numpy as np
import librosa
import pandas as pd
import aggregators
import embeddors
import data_utils
import tensorflow as tf
from tensorflow.keras.layers import Dense, InputLayer
from alibi_detect.cd import MMDDrift
from alibi_detect.cd.preprocess import UAE 
from alibi_detect.utils.kernels import gaussian_kernel

In [2]:
# Change this component to the root of the VOiCES dataset
#DATASET_ROOT = '/Users/jberkowitz/Datasets/VOiCES_devkit'
DATASET_ROOT = '/mnt/fs03/shared/datasets/VOiCES_devkit'
# Convenience function to add root to data path
add_root = lambda x: os.path.join(DATASET_ROOT,x)

First we will load the dataframe for the `training` slice of VOiCES, and perform a few cleaning operations

In [3]:
full_index_df = pd.read_csv(add_root('references/train_index.csv'))
# Drop recordings that don't match in length to source audio
trimmed_index = full_index_df[full_index_df['noisy_length']==full_index_df['source_length']]
trimmed_index.head(2)

,index,chapter,degrees,distractor,filename,gender,mic,query_name,room,segment,source,speaker,transcript,noisy_length,noisy_sr,noisy_time,source_length,source_sr,source_time
0,0,9960,60,musi,distant-16k/speech/train/rm1/musi/sp0083/Lab41...,F,1,Lab41-SRI-VOiCES-rm1-musi-sp0083-ch009960-sg00...,rm1,42,source-16k/train/sp0083/Lab41-SRI-VOiCES-src-s...,83,the horrible glowing tool disappeared into the...,258880,16000,16.18,258880,16000,16.18
1,1,9960,60,musi,distant-16k/speech/train/rm1/musi/sp0083/Lab41...,F,5,Lab41-SRI-VOiCES-rm1-musi-sp0083-ch009960-sg00...,rm1,42,source-16k/train/sp0083/Lab41-SRI-VOiCES-src-s...,83,the horrible glowing tool disappeared into the...,258880,16000,16.18,258880,16000,16.18


Next we initialize the embeddors

In [4]:
w2v = embeddors.Wav2VecEmbeddor(weight_path='models/wav2vec_large.pt')
#w2v = embeddors.Wav2VecEmbeddor(use_cpu=False)
trill=embeddors.TrillEmbeddor()

2020-12-13 12:58:40 | INFO | fairseq.models.wav2vec.wav2vec | Wav2VecModel(
  (feature_extractor): ConvFeatureExtractionModel(
    (conv_layers): ModuleList(
      (0): Sequential(
        (0): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (1): Dropout(p=0.0, inplace=False)
        (2): Fp32GroupNorm(1, 512, eps=1e-05, affine=True)
        (3): ReLU()
      )
      (1): Sequential(
        (0): Conv1d(512, 512, kernel_size=(8,), stride=(4,), bias=False)
        (1): Dropout(p=0.0, inplace=False)
        (2): Fp32GroupNorm(1, 512, eps=1e-05, affine=True)
        (3): ReLU()
      )
      (2): Sequential(
        (0): Conv1d(512, 512, kernel_size=(4,), stride=(2,), bias=False)
        (1): Dropout(p=0.0, inplace=False)
        (2): Fp32GroupNorm(1, 512, eps=1e-05, affine=True)
        (3): ReLU()
      )
      (3): Sequential(
        (0): Conv1d(512, 512, kernel_size=(4,), stride=(2,), bias=False)
        (1): Dropout(p=0.0, inplace=False)
        (2): Fp32GroupNorm

Initialize the aggregators

In [5]:
mean_ag = aggregators.MeanAggregator()
pca_ag = aggregators.PCAAggregator()

We will also use untrained autoencoders (UAE) to do some preliminary dimension reduction, as in [Failing Loudly: An Empirical Study of Methods for Detecting Dataset Shift](https://arxiv.org/pdf/1810.11953.pdf).  We will initialize a UAE for both Trill and Wave2Vec embeddings, as they have different dimensionality. 

In [6]:
tf.random.set_seed(0) # set seed for reproducibility
encoding_dim=10 # fixed dimensionality

w2v_encoder_net = tf.keras.Sequential(
    [
        InputLayer(input_shape=(512,)),
        Dense(100,),
        Dense(encoding_dim,)
    ]
)
w2v_uae = UAE(w2v_encoder_net)

trill_encoder_net = tf.keras.Sequential(
    [
        InputLayer(input_shape=(2048,)),
        Dense(100,),
        Dense(encoding_dim,)
    ]
)
trill_uae = UAE(trill_encoder_net)

Next we will vectorize the data and store it in arrays.  For the clean data and each noise type in VOiCES (none,musi,babb,tele).  We will define a convenience function here to automate the production and storage of all these arrays in dictionaries.

In [7]:
def make_vector_dict(file_list):
    """
    Inputs
        file_list - a list of paths to wav files corresponding to all the waveforms to vectorize.
    Outputs:
        vector_dict - a dictionary with keys w2v_mean,w2v_pca,trill_mean,trill_pca
    """
    waveforms = [librosa.load(file_name,sr=16000)[0] for file_name in file_list]
    vector_dict = {}
    vector_dict['w2v_mean']=data_utils.pipeline(wav_list=waveforms,embeddor=w2v,aggregator=mean_ag)
    vector_dict['w2v_pca']=data_utils.pipeline(wav_list=waveforms,embeddor=w2v,aggregator=pca_ag)
    vector_dict['trill_mean']=data_utils.pipeline(wav_list=waveforms,embeddor=trill,aggregator=mean_ag)
    vector_dict['trill_pca']=data_utils.pipeline(wav_list=waveforms,embeddor=trill,aggregator=pca_ag)
    return vector_dict

In [15]:
# We will save computed vector arrays here for reuse
os.mkdir('precomputed_vectors')

In [24]:
# We vectorize the clean, source speech
source_files = list(map(add_root,trimmed_index['source'].unique()))
source_vector_dict = make_vector_dict(source_files)
pickle.dump(source_vector_dict,open('precomputed_vectors/source_vector_dict.pkl','wb'))

2020-12-12 19:29:43 | WARNING | tensorflow | 5 out of the last 41 calls to <function recreate_function.<locals>.restored_function_body at 0x7efbb8d4a4d0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


2020-12-12 19:29:43 | WARNING | tensorflow | 6 out of the last 42 calls to <function recreate_function.<locals>.restored_function_body at 0x7efbb8d4a4d0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


2020-12-12 19:29:44 | WARNING | tensorflow | 7 out of the last 44 calls to <function recreate_function.<locals>.restored_function_body at 0x7efbb8d4a4d0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


2020-12-12 19:29:44 | WARNING | tensorflow | 8 out of the last 45 calls to <function recreate_function.<locals>.restored_function_body at 0x7efbb8d4a4d0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


2020-12-12 19:29:44 | WARNING | tensorflow | 9 out of the last 46 calls to <function recreate_function.<locals>.restored_function_body at 0x7efbb8d4a4d0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


...

2020-12-12 19:30:56 | WARNING | tensorflow | 6 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7efbb8d4a4d0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


2020-12-12 19:30:56 | WARNING | tensorflow | 7 out of the last 12 calls to <function recreate_function.<locals>.restored_function_body at 0x7efbb8d4a4d0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


2020-12-12 19:30:56 | WARNING | tensorflow | 7 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7efbb8d4a4d0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


2020-12-12 19:30:57 | WARNING | tensorflow | 6 out of the last 11 calls to <function recreate_function.<locals>.restored_function_body at 0x7efbb8d4a4d0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


312.86125111579895


This next cell unfortunately takes several hours to run, even with a GPU

In [34]:
#distractor type none
t1 = time.time()
none_files = list(map(add_root,trimmed_index[trimmed_index['distractor']=='none']['filename'].tolist()))
none_vector_dict = make_vector_dict(none_files)
pickle.dump(none_vector_dict,open('precomputed_vectors/none_vector_dict.pkl','wb'))
t2 = (time.time()-t1)/60.0
print('done with distractor type none in {} minutes'.format(t2))

#distractor type musi
t1 = time.time()
musi_files = list(map(add_root,trimmed_index[trimmed_index['distractor']=='musi']['filename'].tolist()))
musi_vector_dict = make_vector_dict(musi_files)
pickle.dump(musi_vector_dict,open('precomputed_vectors/musi_vector_dict.pkl','wb'))
t2 = (time.time()-t1)/60.0
print('done with distractor type musi in {} minutes'.format(t2))

#distractor type tele
t1 = time.time()
tele_files = list(map(add_root,trimmed_index[trimmed_index['distractor']=='tele']['filename'].tolist()))
tele_vector_dict = make_vector_dict(tele_files)
pickle.dump(tele_vector_dict,open('precomputed_vectors/tele_vector_dict.pkl','wb'))
t2 = (time.time()-t1)/60.0
print('done with distractor type tele in {} minutes'.format(t2))

#distractor type babb
t1 = time.time()
babb_files = list(map(add_root,trimmed_index[trimmed_index['distractor']=='babb']['filename'].tolist()))
babb_vector_dict = make_vector_dict(babb_files)
pickle.dump(babb_vector_dict,open('precomputed_vectors/babb_vector_dict.pkl','wb'))
t2 = (time.time()-t1)/60.0
print('done with distractor type babb in {} minutes'.format(t2))

done with distractor type none in 35.62715365886688 minutes
done with distractor type musi in 35.89223426977794 minutes
done with distractor type tele in 35.549998088677725 minutes
done with distractor type babb in 35.54479220708211 minutes


In [8]:
# Cell for reloading pre-computed arrays
source_vector_dict = pickle.load(open('precomputed_vectors/source_vector_dict.pkl','rb'))
none_vector_dict = pickle.load(open('precomputed_vectors/none_vector_dict.pkl','rb'))
musi_vector_dict = pickle.load(open('precomputed_vectors/musi_vector_dict.pkl','rb'))
tele_vector_dict = pickle.load(open('precomputed_vectors/tele_vector_dict.pkl','rb'))
babb_vector_dict = pickle.load(open('precomputed_vectors/babb_vector_dict.pkl','rb'))

Next we will carry out [two-sample nonparametric hypothesis testing](https://en.wikipedia.org/wiki/Two-sample_hypothesis_testing) with the [Kernel two-sample test (MMD)](https://en.wikipedia.org/wiki/Kernel_embedding_of_distributions#Kernel_two-sample_test).  We will use the implementation of the MMD test provided in the [alibi-detect library](https://docs.seldon.io/projects/alibi-detect/en/stable/methods/mmddrift.html).

In [17]:
cd = MMDDrift(
    p_val=0.05,
    X_ref = source_vector_dict['w2v_mean'],
    preprocess_kwargs={'model':w2v_uae,'batch_size':128},
    kernel=gaussian_kernel,
    chunk_size=100,
    n_permutations=20
)

In [18]:
cd.predict(none_vector_dict['w2v_mean'],return_p_val=True)

/home/jberkowitz/anaconda3/envs/audio_shift/lib/python3.7/site-packages/dask/array/core.py:3871: PerformanceWarning: Increasing number of chunks by factor of 32
  **blockwise_kwargs,
/home/jberkowitz/anaconda3/envs/audio_shift/lib/python3.7/site-packages/dask/array/core.py:3871: PerformanceWarning: Increasing number of chunks by factor of 32
  **blockwise_kwargs,
/home/jberkowitz/anaconda3/envs/audio_shift/lib/python3.7/site-packages/dask/array/core.py:3871: PerformanceWarning: Increasing number of chunks by factor of 32
  **blockwise_kwargs,
/home/jberkowitz/anaconda3/envs/audio_shift/lib/python3.7/site-packages/dask/array/core.py:3871: PerformanceWarning: Increasing number of chunks by factor of 32
  **blockwise_kwargs,
/home/jberkowitz/anaconda3/envs/audio_shift/lib/python3.7/site-packages/dask/array/core.py:3871: PerformanceWarning: Increasing number of chunks by factor of 32
  **blockwise_kwargs,
/home/jberkowitz/anaconda3/envs/audio_shift/lib/python3.7/site-packages/dask/array/co

{'data': {'is_drift': 1,
  'distance': 0.3473879,
  'p_val': 0.0,
  'threshold': 0.05},
 'meta': {'name': 'MMDDrift', 'detector_type': 'offline', 'data_type': None}}